In [21]:
import time
import pandas as pd
import os
import sys
import random
import subprocess
import matplotlib.pyplot as plt
from IPython.core.display import HTML
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2



CB5_DIR = os.path.dirname(os.getcwd())
SIM_DIR=os.path.join(CB5_DIR,"sim")
SCRIPT_DIR = os.path.join(CB5_DIR,"scripts")
TRACES =os.path.join(CB5_DIR,"traces")
RESULT_DIR=os.path.join(CB5_DIR, "results")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# create a lists of workloads
workloads_83 = []
workloads_12 = []

all_works = sorted(os.listdir(TRACES))
length = ["LONG", "SHORT"]
types = ["MOBILE", "SERVER"]
for l in length:
    for t in types:
        temp_list = []
        for w in all_works:
            if (l + "_" + t) in w:
                temp_list.append(w)
        t10_list = random.sample(temp_list , k=min(3,len(temp_list)))
        temp_list = random.sample(temp_list , k=min(25,len(temp_list)))
        for _t in temp_list:
            workloads_83.append(_t)
        for _t in t10_list:
            workloads_12.append(_t)
            


In [23]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [24]:
policies = ["GHRP", "RANDOM", "LRU", "RRIP"]

cmd_base = [os.path.join(SIM_DIR,"predictor")]
# os.mkdir(os.path.join(RESULT_DIR,"12_works"))
print(workloads_12)
f = open(SCRIPT_DIR + "/12_works.sh", "w")
temp = os.path.join(RESULT_DIR,"exp_4bits")
for i in range(len(workloads_12)//2 + 1):
    for j in range(2):
        if (i*2+j < len(workloads_12)):
            for pol in policies:
                mycommand = cmd_base[:]
                mycommand.append(os.path.join(TRACES, workloads_12[i*2+j]))
                mycommand.append(pol)
                mycommand.append(">")
                a = mycommand[1].split("/")
                mycommand.append((temp +"/"+ a[-1] + mycommand[2] +".txt"))
                mycommand.append("&")
                mycommand = " ".join(mycommand)
                f.write(mycommand)
                f.write("\n")
    f.write("for pid in $(jobs -p)\n")
    f.write("do\n")
    f.write("\twait $pid\n")
    f.write("done\n")
        
f.close()              
#     print(cmds)
#     procs = [ subprocess.Popen(i,  stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True) for i in cmds ]


#     temp = os.path.join(RESULT_DIR,"83_works")
#     for i in range(len(procs)):
#         print("*****************************************************")
#         p = procs[i]
        
#         a =  cmds[i][1].split("/")
#         print(temp +"/"+ a[-1] + cmds[i][2] +".txt", "w")
#         out, err = p.communicate()
        
#         f = open(temp +"/"+ a[-1] + cmds[i][2] +".txt", "w")
#         f.write(out)
#         f.close()
#         print(out)
#         print(err)

['LONG_MOBILE-11.bt9.trace.gz', 'LONG_MOBILE-29.bt9.trace.gz', 'LONG_MOBILE-17.bt9.trace.gz', 'LONG_SERVER-7.bt9.trace.gz', 'LONG_SERVER-8.bt9.trace.gz', 'LONG_SERVER-2.bt9.trace.gz', 'SHORT_MOBILE-4.bt9.trace.gz', 'SHORT_MOBILE-44.bt9.trace.gz', 'SHORT_MOBILE-88.bt9.trace.gz', 'SHORT_SERVER-272.bt9.trace.gz', 'SHORT_SERVER-113.bt9.trace.gz', 'SHORT_SERVER-254.bt9.trace.gz']


In [31]:
# cmds = [ ]
# cmd_base = ["echo"]
# for i in range(len(workloads_12)//10):
#     for j in range(10):
#         if (i*10+j < len(workloads_12)):
#             mycommand = cmd_base[:]
#             mycommand.append(os.path.join(TRACES, workloads_12[i*10+j]))
#             cmds.append(mycommand)
            
# print(cmds)
# procs = [ subprocess.Popen(i,  stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True) for i in cmds ]


# for i in range(len(procs)):
#     print("*****************************************************")
#     p = procs[i]
#     out, err = p.communicate()
#     f = open((RESULT_DIR + ("/"+workloads_12[i] +".txt")),"w")
#     f.write(out)
#     f.close()
#     print(out)
#     print(err)

[['echo', '/home/shyngys/cbp2016.eval/traces/LONG_MOBILE-5.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/LONG_MOBILE-3.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/LONG_MOBILE-22.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/LONG_SERVER-4.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/LONG_SERVER-8.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/LONG_SERVER-6.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/SHORT_MOBILE-69.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/SHORT_MOBILE-7.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/SHORT_MOBILE-3.bt9.trace.gz'], ['echo', '/home/shyngys/cbp2016.eval/traces/SHORT_SERVER-85.bt9.trace.gz']]
*****************************************************
/home/shyngys/cbp2016.eval/traces/LONG_MOBILE-5.bt9.trace.gz


*****************************************************
/home/shyngys/cbp2016.eval/traces/LONG_MOBILE-3.bt9.trace.gz


******************

In [33]:
workloads_12

['LONG_MOBILE-5.bt9.trace.gz',
 'LONG_MOBILE-3.bt9.trace.gz',
 'LONG_MOBILE-22.bt9.trace.gz',
 'LONG_SERVER-4.bt9.trace.gz',
 'LONG_SERVER-8.bt9.trace.gz',
 'LONG_SERVER-6.bt9.trace.gz',
 'SHORT_MOBILE-69.bt9.trace.gz',
 'SHORT_MOBILE-7.bt9.trace.gz',
 'SHORT_MOBILE-3.bt9.trace.gz',
 'SHORT_SERVER-85.bt9.trace.gz',
 'SHORT_SERVER-97.bt9.trace.gz',
 'SHORT_SERVER-203.bt9.trace.gz']